En este notebook se realizan las consultas necesarias a cada set de datos para generar un set de datos óptimo para cada funcion de la API.

---

# Importamos las librerias necesarias.

In [14]:
import pandas as pd # Trabajamos datos con pandas.
import ast # Evaluaremos una cadena de texto que puede ser un iterable con ast.

Defino donde se van a guardar los datos

In [15]:
apipath = '../Api/ApiData/'

# 1. developer( )

In [16]:
# Cargo el data frame y las columnas que usaremos.
df = pd.read_csv('../Data/Processed/games_api.csv', dtype={'year':'str', 'id':'str'}, usecols=['price', 'id', 'developer', 'year'])
# Agrupamos por desarrollador y año, para facilitar la búsqueda de la API. Contamos los id y contamos la cantidad de precios que sean 0 (free to play).
df = df.groupby(['developer', 'year']).agg({'id':'count', 'price': lambda x: (x == 0).sum()}).reset_index()
# Dividimos la cantidad total de juegos gratis sobre la cantidad total de juegos para obtener el porcentaje de gratis.
df['price'] = (df['price'] / df['id'])*100
# Formateamos el porcentaje para una mejor lectura.
df['price'] = df['price'].apply(lambda x: f"{x:.2f} %")
# Renombramos las columnas para una mejor lectura.
df = df.rename(columns={'id':'Cantidad', 'price':'% free to play'})
# Visualizamos el resultado.
df.tail()

,developer,year,Cantidad,% free to play
14700,萌石游戏,2017.0,1,0.00 %
14701,高考恋爱委员会,2015.0,1,100.00 %
14702,"高考恋爱委员会,Days",2015.0,1,0.00 %
14703,"高考恋爱委员会,橘子班",2015.0,1,0.00 %
14704,"高考恋爱委员会,橘子班",2017.0,1,0.00 %


Exportamos los datos

In [17]:
# Volvemos la columna objetivo minúscula para una mejor experiencia de búsqueda.
df['developer'] = df['developer'].str.lower()
df.to_csv(apipath + 'developer.csv', index=False)

# 2. userdata( )

In [18]:
# Cargamos los datos que usaremos.
df_users = pd.read_csv('../Data/Processed/items_api.csv', usecols=['id', 'item_id'])
df_recommend = pd.read_csv('../Data/Processed/df_reviews.csv', usecols=['id', 'recommend'])
df_money = pd.read_csv('../Data/Processed/games_api.csv', usecols=['id', 'price'])
# Empezamos mezclando los usuarios con la información precio del juego que tengan.
df = pd.merge(df_users, df_money, left_on='item_id', right_on='id', suffixes=('_user', '_item'), how='inner')
# Eliminamos la columna restante.
df = df.drop(columns=['id_item'])
# Agrupamos por id de usuario y sumamos la cantidad de items y el total de dinero gastado.
df = df.groupby('id_user').agg({'item_id':'count', 'price':'sum'})
# Cambiamos el nombre de las columnas.
df = df.rename(columns={'item_id':'Cantidad_items', 'price':'Dinero_gastado'})
# creamos una columna para que cuente el total de registros.
df_recommend['total'] = df_recommend['recommend']
# Agrupamos y sumamos la cantidad de recomendaciones (al ser un booleano los True contaran como 1 y los False como 0) y a su vez contamos los registros.
recommends = df_recommend.groupby('id').agg({'recommend':'sum', 'total':'count'})
# Agregamos el porcentaje de recomendación al dataframe, dividiendo el total de recomendados sobre el total, y multiplicándolo por 100.
df['recommend'] = (recommends['recommend'] / recommends['total'])*100
# Formateamos la columna para una mejor lectura.
df['recommend'] = df['recommend'].apply(lambda x: f"{x:.2f} %" if not pd.isnull(x) else x)
# Llenamos los valores que no tenían coincidencias con No hay reseñas. No los llené con 0 porque se puede malinterpretar como un usuario que tiene reseñas pero en ninguna recomienda.
df = df.fillna('No hay reseñas.')
# Reinicio el indice por comodidad para la API.
df = df.reset_index()
# Previsualizamos.
df.head()

,id_user,Cantidad_items,Dinero_gastado,recommend
0,--000--,48,402.77,No hay reseñas.
1,--ace--,38,166.82,100.00 %
2,--ionex--,19,109.92,100.00 %
3,-2SV-vuLB-Kg,53,437.49,100.00 %
4,-404PageNotFound-,112,1514.31,No hay reseñas.


Exportamos los datos

In [19]:
# En este caso no volvemos la columna objetivo minúsculas, ya que los usuarios deben tener IDs únicos, y esto puede interferir en eso.
df.to_csv(apipath + 'user_data.csv', index=False)

# 3. UserForGenre( )

In [20]:
# Cargamos los datos.
df_games = pd.read_csv('../Data/Processed/games_api.csv', usecols=['id', 'genres', 'year'])
df_users = pd.read_csv('../Data/Processed/items_api.csv', usecols=['id', 'item_id', 'playtime_forever'])
# Eliminamos los faltantes previamente conservados ( revisar notebook ETL-Games.ipynb para más información ).
df_games.dropna(inplace=True)
# Eliminamos los tiempos de juego 0 ya que solo aportaran carga computacional a las funciones de pandas.
df_users = df_users[df_users['playtime_forever'] > 0]
# Convertimos la columna de géneros a un iterable.
df_games['genres'] = df_games['genres'].apply(ast.literal_eval)
# Mezclamos los dataframes en id para obtener juntar el tiempo de juego con cada juego. También eliminamos la columna repetida id.
merged = pd.merge(df_games, df_users, left_on='id', right_on='item_id', how='inner').drop(columns=['id_x']).rename(columns={'id_y':'user_id'})
# Extraemos los géneros para que sean filas.
genres = merged.explode('genres')
# Agrupamos las filas de géneros extraídas por usuario y por año, sumando el tiempo de juego total.
df_year_time = genres.groupby(['genres', 'user_id', 'year']).agg({'playtime_forever': 'sum'}).reset_index()
# Agrupamos nuevamente pero esta vez sin el año, para saber el usuario con más horas totales para cada género.
df = genres.groupby(['genres', 'user_id']).agg({'playtime_forever': 'sum'}).reset_index()
# Extraemos para cada genero, el usuario con más horas acumuladas para ese género. Lo ordenamos para una mejor previsualización.
df_max = df.loc[df.groupby('genres')['playtime_forever'].idxmax()].sort_values('playtime_forever', ascending=False)
# Filtramos el df con la información de los años, para que contenga a los jugadores que se necesitan (los del top de cada género).
dff = pd.merge(df_year_time, df_max, on=['user_id', 'genres']).drop(columns=['playtime_forever_y'])
# Previsualizamos el dataframe final.
dff.head()

,genres,user_id,year,playtime_forever_x
0,1980s,76561198042170355,2015.0,10467
1,1990's,76561197995146150,2011.0,15820
2,1990's,76561197995146150,2012.0,122
3,2D,76561198083612270,2012.0,24664
4,3D Platformer,76561197995146150,2011.0,15820


Exportamos los datos.

In [21]:
# Volvemos la columna objetivo minúscula para una mejor experiencia de búsqueda.
dff['genres'] = dff['genres'].str.lower()
dff.to_csv(apipath + 'user_for_genre.csv', index=False)

# 4. best_developer_year( )

In [22]:
# Cargo los dataframes junto con las columnas que usaré.
df_devs = pd.read_csv('../Data/Processed/games_api.csv', usecols=['id', 'developer'])
df_recommend = pd.read_csv('../Data/Processed/df_reviews.csv', usecols=['item_id', 'recommend', 'sentiment', 'year'], dtype={'year':str})

# Mezclamos el df en el que se encuentran los desarrolladores, junto con el que se encuentran las calificaciones junto al año en general de los usuarios.
df = pd.merge(df_devs, df_recommend, left_on='id', right_on='item_id', how='inner')
# En la columna sentimiento, le restamos 1 al valor, para que cuando sea positiva (antes 2) valga por 1, cuando sea neutral (antes 1), valga 0, y cuando sea negativa (antes 0), reste 1.
df['sentiment'] = df['sentiment'].apply(lambda x: x - 1)
# Agrupamos por año, y por desarrollador, aplicando la suma a sus puntajes en ambas columnas.
df = df.groupby(['year', 'developer']).agg({'recommend':'sum', 'sentiment':'sum'}).reset_index()
# Obtenemos el total de puntos.
df['total'] = df['recommend'] + df['sentiment']
# Eliminamos las columnas ya que solo usaremos el total.
df = df.drop(columns=['recommend', 'sentiment'])
# Ordenamos el DataFrame por año y total en orden descendente.
df = df.sort_values(by=['year', 'total'], ascending=False)
# Creamos una lista vacía para almacenar los resultados de los 3 mejores desarrolladores de cada año.
top_devs = []
# Con year iterando en cada año único en el DataFrame, podemos entrar en cada año.
for year in df['year'].unique():
    # Seleccionamos los datos correspondientes al año actual.
    df_temp = df[df['year'] == year]
    # Obtenemos los 3 mejores desarrolladores del año basándonos en el total y los agregamos a la lista.
    top_devs.append(df_temp.nlargest(3, 'total'))
# Creamos un DataFrame con los datos de la lista.
top_devs = pd.concat(top_devs, ignore_index=True)
# Visualizamos el DataFrame resultante.
top_devs
# Ya que la columna total no la usaremos en la API, podríamos borrarla, pero en caso de que ocurra que al cargar el DataFrame resultante
# no se cargue en su orden original, prefiero mantener la columna para luego sortear el DataFrame y evitar errores, el cargo de esta tarea es mínimo igualmente.

,year,developer,total
0,2015,Valve,2240
1,2015,Facepunch Studios,477
2,2015,Smartly Dressed Games,258
3,2014,Valve,3095
4,2014,Facepunch Studios,1121
5,2014,Bohemia Interactive,554
6,2013,Valve,1252
7,2013,Facepunch Studios,407
8,2013,Bohemia Interactive,152
9,2012,Valve,419


Exportamos los datos.

In [23]:
top_devs.to_csv(apipath + 'best_developer_year.csv', index=False)

# 5.developer_reviews_analysis( )

In [24]:
# Cargo los dataframes junto con las columnas que usaré.
df_devs = pd.read_csv('../Data/Processed/games_api.csv', usecols=['id', 'developer'])
df_recommend = pd.read_csv('../Data/Processed/df_reviews.csv', usecols=['item_id', 'sentiment'])
# Mezclamos el DataFrame de los developer y sus juegos, junto con sus correspondientes reseñas.
df = pd.merge(df_devs, df_recommend, left_on='id', right_on='item_id', how='inner')
# Eliminamos los sentimientos neutros, ya que solo necesitamos contar los positivos y negativos.
df = df[df['sentiment'] != 1]
# Identificamos los sentimientos positivos.
df['pos'] = df['sentiment'].apply(lambda x: True if x else False)
# Negamos la columna positiva para tener los negativos.
df['neg'] = ~(df['pos'])
# Sumamos sus respectivos totales para obtener el DataFrame final.
df = df.groupby('developer').agg({'pos':'sum', 'neg':'sum'}).reset_index()
# Previsualizamos.
df.head()

,developer,pos,neg
0,10tons Ltd,1,0
1,11 bit studios,20,27
2,14° East,1,0
3,16bit Nights,0,1
4,17-BIT,1,0


Exportamos los datos.

In [25]:
# Volvemos la columna objetivo minúscula para una mejor experiencia de búsqueda.
df['developer'] = df['developer'].str.lower()
df.to_csv(apipath + 'developer_reviews_analysis.csv', index=False)